#**First Deep Reinforcement Learning Algorithm: Deep Q Learning**

#**Human-Like Artificial Intelligence for Atari Games** 

#**Inference Program**

#**Open AI Gym Toolkit**

In [15]:
import sys
import timeit
import os
import os.path
import platform
colab_active = 'google.colab' in sys.modules
print("Colab Active: ", colab_active)
# Operating System
OS = platform.system()                           # returns 'Windows', 'Linux', etc

Colab Active:  True


#**Import of all the needed Atari Roms**

In [16]:
if OS == "Linux":
  if not os.path.exists("./Roms.rar"):
    os.system("wget http://www.atarimania.com/roms/Roms.rar") 
    os.system("unrar x -o+ ./Roms.rar >/dev/null")
    os.system("python -m atari_py.import_roms ./ROMS >/dev/null")

#**Gym Update**

In [17]:
os.system("pip install gym==0.17.3")

if OS == "Linux":
  os.system("pip install gym[atari]")
elif OS == "Windows": 
  os.system("pip install -f https://github.com/Kojoley/atari-py/releases atari_py")     # to make it work on windows

if OS == "Linux":
  os.system("pip install pyvirtualdisplay > /dev/null 2>&1")
  os.system("apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1")

  os.system("apt-get update > /dev/null 2>&1")
  os.system("apt-get install cmake > /dev/null 2>&1")
  os.system("pip install --upgrade setuptools 2>&1")
  os.system("pip install ez_setup > /dev/null 2>&1")

#**All General Imports**

In [18]:
import numpy as np
import glob
import io
import os.path
import base64
import math
import random
import gym
from gym import logger as gymlogger
from gym.wrappers import Monitor
import numpy as np
from collections import deque

if colab_active is True: 
  from pyvirtualdisplay import Display
  Display().start()

  from IPython import display
  from IPython.display import HTML
  from IPython import display as ipythondisplay
  import matplotlib
  import matplotlib.pyplot as plt
  %matplotlib inline

#**All Tensorflow 2.0 Imports**

In [19]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import model_from_json

#**All Globals**

In [20]:
env_name = "MsPacman-v0"
#num_episodes = 500   # 7 days
num_episodes = 1
num_timesteps = 20000
summary = False
epsilon = 0.8   

#**All Initializations**

In [21]:
done = False
time_step = 0

#**All Functions Definitions**

In [22]:
def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay loop controls style="height: 400px;">
                                        <source src="data:video/mp4;base64,{0}" type="video/mp4" />
                                        </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    
def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

def preprocess_state(state):
    image = state[1:176:2, ::2]
    image = image.mean(axis=2)
    image[image==color] = 0
    image = (image - 128) / 128 - 1
    image = np.expand_dims(image.reshape(88, 80, 1), axis=0)

    return image

def epsilon_greedy(dqn, state, action_size, epsilon):
    if random.uniform(0,1) < epsilon:
        return np.random.randint(action_size)
    Q_values = dqn.predict(state)
    
    return np.argmax(Q_values[0])

#**All Initializations**

In [23]:
done = False
time_step = 0

if colab_active is True: 
  gymlogger.set_level(40)                              #error only
  display = Display(visible=0, size=(1400, 900))
  display.start()

state_size = (88, 80, 1)
color = np.array([210, 164, 74]).mean()

if colab_active is True: 
  env = wrap_env(gym.make(env_name))
else:
  env = gym.make(env_name)

action_size = env.action_space.n
print(env.action_space)

Discrete(9)


#**Mounting Google Drive via Code**

In [24]:
if colab_active is True: 
  from google.colab import drive
  drive.mount('/content/drive')
  sys.path.append('/content/drive/My Drive')
  model_path = "/content/drive/MyDrive/DL_Models/"
  if os.path.isdir(model_path) is False: 
    os.makedirs(model_path) 
else: 
  model_path = "./"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#**Model Initialization**

In [25]:
with open(model_path+ env_name.lower()+"_deep_q_network.json", 'r') as json_file:
    loaded_model_json = json_file.read()

deep_q_network = model_from_json(loaded_model_json)
deep_q_network.load_weights(model_path+ env_name.lower()+"_deep_q_network.h5")
print("Atari Deep Q Network Loaded Successfully...")

Atari Deep Q Network Loaded Successfully...


#**Smart Trained Policy**

In [28]:
Return = 0
state = preprocess_state(env.reset())
while True:
    env.render()

    action = epsilon_greedy(deep_q_network, state, action_size, epsilon)

    state, reward, done, info = env.step(action) 
    state = preprocess_state(state)
    Return += reward

    if done: 
      print('Return', Return)
      break;
env.close()
if colab_active is True: 
  show_video()

Return 120.0
